In [4]:
# install mysql connector: pip install mysql-connector-python
import mysql.connector
import pandas as pd
# used to suppress duplicates error
from mysql.connector import Error

In [5]:
# specify file directory
# excel files will have issue, have to convert to csv with utf-8, with delimiter, comma separation may come out weird, had to play with this when reading csv
directory = "/Users/allison.lu/OneDrive - University of Florida/app_data.csv"
df = pd.read_csv(directory)

# dropped date since it is incomplete
df = df.drop('Privacy Policy Date', axis=1)
df = df.drop_duplicates()
display(df)

,Application Name,Application ID,Category,Downloads,Developer Name,Version,Privacy Policy URL
0,WhatsApp Messenger,com.whatsapp,Communication,7522116811,WhatsApp LLC,2.22.22.80,http://www.whatsapp.com/legal/#Privacy
1,Facebook,com.facebook.katana,Social,8158681556,"Meta Platforms, Inc.",Varies with device,https://www.facebook.com/about/privacy/
2,Instagram,com.instagram.android,Social,4470353588,Instagram,Varies with device,http://instagram.com/legal/privacy/
3,TikTok,com.zhiliaoapp.musically,Social,2324399107,TikTok Pte. Ltd.,Varies with device,https://www.tiktok.com/legal/privacy-policy
4,Snapchat,com.snapchat.android,Communication,2240707819,Snap Inc,12.05.0.37,http://www.snapchat.com/privacy
...,...,...,...,...,...,...,...
10841,Explore Daniel's Neighborhood,org.pbskids.dtigerexploreneighborhood,Educational,103010,PBS KIDS,4.5.9,https://pbskids.org/privacy/
10842,Molly of Denali: Learn about N,org.pbskids.mollyofdenali,Educational,90613,PBS KIDS,1.1.0,http://www.pbskids.org/privacy
10843,Daniel Tiger Grr-ific Feelings,org.pbskids.dtgrrificfeelings,Education,76744,PBS KIDS,2,http://pbskids.org/privacy/
10844,Daniel Tiger: Play at Home,org.pbskids.dtigplayhomeuni,Education,6624,PBS KIDS,3.0.1,http://pbskids.org/privacy


In [9]:
# change password and database to your own password and database
# localhost and root should still work

db = mysql.connector.connect(
  host = "localhost",
  user = "root",
  passwd = "password",
  database = "database"
)

tpls = [tuple(x) for x in df.to_numpy()]

# columns need to be in same order as df
cols = "app_name, app_id, category, downloads, developer_name, version, policy_url"

# add ignore for duplicate columns, can take care of duplicates later directly in database
# may have to change policy_md to name of your table
query = "INSERT IGNORE INTO policy_md(%s) VALUES (%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

# dup_query = "DELETE t1 FROM priv_policy.policy_md t1, priv_policy.policy_md t2 WHERE t1.id > t2.id AND t1.app_id = t2.app_id;"

cur = db.cursor()

try:
    cur.executemany(query, tpls)
#     cur.execute(dup_query)
    db.commit()
    print("Data inserted using execute_many() successfully...")
except Error as e:
    print("Error while inserting to MySQL", e)
    cur.close()


cur.close()
db.close()


Data inserted using execute_many() successfully...


In [26]:
# tpls = [tuple(x) for x in df.to_numpy()]
# cols = ','.join(list(df.columns))
# print(cols)


Application Name,Application ID,Category,Downloads,Developer Name,Version,Privacy Policy URL
